### Train a bilstm to generate jokes in forward and reverse based on a controlled bit

In [1]:
import pickle
import importlib
from library import data_preprocess as dp
importlib.reload(dp)
import random
from time import time

import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM, Bidirectional
from keras.layers import Embedding, TimeDistributed, Flatten, Merge, Concatenate
from keras import regularizers
from keras.metrics import sparse_categorical_accuracy, sparse_categorical_crossentropy
from keras.models import load_model
from keras.optimizers import Adam
from keras.models import Model

from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import tensorflow as tf
from keras import backend as K

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Variables

In [2]:
# http://nlp.stanford.edu/data/glove.6B.zip
DATA_PATH = './datasets/jokes.pickle'
VOCAB_PATH = './datasets/jokes_vocabulary.pickle'
MODELS_PATH = './models/'

GLOVE_PATH = './data/glove.6B.200d.txt'

MODEL_PREFIX = 'jokes_controlled_stacked_lstm_glove'

MAX_SEQUENCE_LENGTH = 13
VALIDATION_SPLIT = 0.2

GLOVE_EMBEDDING_DIM = 200
EMBEDDING_DIM1 = 256
EMBEDDING_DIM2 = 256
HIDDEN_DIM1 = 512
HIDDEN_DIM2 = 256
DEEPER_DIM = 256
DROPOUT_FACTOR = 0.2
REGULARIZATION = 0.00001
LEARNING_RATE = 0.003

DATA_PERCENT = 0.1

RUN_INDEX = 1

In [3]:
with open(DATA_PATH, 'rb') as pickleFile:
    sentences = pickle.load(pickleFile)

with open(VOCAB_PATH, 'rb') as pickleFile:
    vocab = pickle.load(pickleFile)
    
random.shuffle(sentences)

print("Number of sentences = ", len(sentences))
print(sentences[:2])
print("Vocab size = ", len(vocab))
print(vocab[:10])

Number of sentences =  96910
['sos if god made everything threedots is god chinese ? eos', 'sos before you refer to someone as your ex , make sure they know you dated . eos']
Vocab size =  8922
['sos', 'did', 'you', 'hear', 'about', 'the', 'new', 'corduroy', 'pillows', '?']


In [4]:
# tokenize data
num_words = len(vocab)

tokenizer = Tokenizer(num_words=None, filters='', lower=True, split=' ', 
                      char_level=False, oov_token=None)
tokenizer.fit_on_texts(sentences)
assert num_words == len(tokenizer.word_index)

encoded_sentences = tokenizer.texts_to_sequences(sentences)
print(encoded_sentences[:5])

VOCAB_SIZE = len(tokenizer.word_index) + 1
print(VOCAB_SIZE)

[[1, 38, 256, 205, 332, 15, 18, 256, 541, 6, 2], [1, 159, 9, 5211, 8, 125, 93, 35, 493, 10, 88, 249, 37, 67, 9, 2747, 3, 2], [1, 38, 9, 140, 752, 30, 983, 26, 4, 559, 15, 15, 37, 95, 27, 8, 1406, 9, 29, 904, 42, 5353, 2], [1, 217, 23, 17, 36, 5, 372, 60, 34, 61, 201, 123, 61, 28, 70, 24, 451, 6, 4, 23, 11, 18, 16, 602, 6, 11, 2], [1, 5, 704, 1294, 12, 924, 15, 5, 704, 1294, 12, 924, 188, 72, 4, 97, 3, 16, 24, 1761, 3, 2]]
8923


In [9]:
# saving
with open(MODELS_PATH + MODEL_PREFIX + '_tokenizer_' + str(RUN_INDEX) + '.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Preparing Training Data

In [5]:
TAG_SIZE = 2
X_data = []
y_data = []
for sentence in encoded_sentences:
    l = len(sentence)
    sliding_window_length = min(l-3, MAX_SEQUENCE_LENGTH)
    step_size = 1
    for i in range(0, l - sliding_window_length, step_size):
        X_data.append(sentence[i:i+sliding_window_length])
        y_data.append(sentence[i+1:i+sliding_window_length+1])
        
print("Total training data size = ", len(X_data))
MAX_SEQ_LEN = max([len(seq) for seq in X_data])
print("Max seq len = ", MAX_SEQ_LEN)

forward_X_data = pad_sequences(X_data, maxlen=MAX_SEQ_LEN, padding='pre')
forward_y_data = pad_sequences(y_data, maxlen=MAX_SEQ_LEN, padding='pre').reshape(-1, MAX_SEQ_LEN, 1)
forward_tag = to_categorical(np.full((forward_X_data.shape[0], MAX_SEQ_LEN), 0), TAG_SIZE)

#print(forward_X_data.shape)
#print(forward_X_data[:2])
#print(forward_y_data.shape)
#print(forward_y_data[:2])
#print(forward_tag.shape)

reverse_X_data = pad_sequences([item[::-1] for item in y_data], maxlen=MAX_SEQ_LEN, padding='pre')
reverse_y_data = pad_sequences([item[::-1] for item in X_data], maxlen=MAX_SEQ_LEN, padding='pre').reshape(-1, MAX_SEQ_LEN, 1)
reverse_tag = to_categorical(np.full((reverse_X_data.shape[0], MAX_SEQ_LEN), 1), TAG_SIZE)

#print(reverse_X_data.shape)
#print(reverse_X_data[:2])
#print(reverse_y_data.shape)
#print(reverse_y_data[:2])
#print(reverse_tag.shape)

X_data = np.concatenate((forward_X_data, reverse_X_data), axis=0)
y_data = np.concatenate((forward_y_data, reverse_y_data), axis=0)
tag_data = np.concatenate((forward_tag, reverse_tag), axis=0)

# shuffle
perm = np.random.permutation(X_data.shape[0])
X_data = X_data[perm]
y_data = y_data[perm]
tag_data = tag_data[perm]
print(X_data.shape)
print(X_data[:4])
print(y_data.shape)
print(y_data[:4])
print(tag_data.shape)
print(tag_data[:4])

Total training data size =  833131
Max seq len =  13
(1666262, 13)
[[ 150  246    5  194  128 2544  297   95   16   41   15   82  560]
 [ 169    4   24   31   15 7256 5006  243  726    5    8  190  633]
 [   0   36    9   98    6   51    5  663   28 2007   53  141    6]
 [ 636    7  134   46    5   11   23   22   11    6 4588   17   11]]
(1666262, 13, 1)
[[[ 246]
  [   5]
  [ 194]
  [ 128]
  [2544]
  [ 297]
  [  95]
  [  16]
  [  41]
  [  15]
  [  82]
  [ 560]
  [   4]]

 [[   4]
  [  24]
  [  31]
  [  15]
  [7256]
  [5006]
  [ 243]
  [ 726]
  [   5]
  [   8]
  [ 190]
  [ 633]
  [   4]]

 [[   0]
  [   9]
  [  98]
  [   6]
  [  51]
  [   5]
  [ 663]
  [  28]
  [2007]
  [  53]
  [ 141]
  [   6]
  [  15]]

 [[   7]
  [ 134]
  [  46]
  [   5]
  [  11]
  [  23]
  [  22]
  [  11]
  [   6]
  [4588]
  [  17]
  [  11]
  [  23]]]
(1666262, 13, 2)
[[[0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 1.]
  [0. 1.

In [7]:
print('Indexing glove word vectors')
#Glove Vectors
glove_embeddings_index = {}
f = open(GLOVE_PATH)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove_embeddings_index[word] = coefs
f.close()
print('Total %s word vectors.' % len(glove_embeddings_index))

Indexing glove word vectors
Total 400000 word vectors.


In [8]:
print('Preparing glove embedding matrix')
glove_embedding_matrix = np.zeros((VOCAB_SIZE, GLOVE_EMBEDDING_DIM))
for word,i in tokenizer.word_index.items():
    embedding_vector = glove_embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        glove_embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(glove_embedding_matrix, axis=1) == 0))
print(glove_embedding_matrix.shape)

Preparing glove embedding matrix
Null word embeddings: 255
(8923, 200)


In [22]:
# define model
def StackedLSTM(vocab_size, glove_embedding_dim, glove_embedding_matrix, embedding_dim1, embedding_dim2,
           hidden_dim1, hidden_dim2, deeper_dim, max_seq_len, tag_size,
           dropout_factor=0.5, regularization=0.00001, learning_rate=0.001):
    
    inputs = Input(shape=(None,))
    tag_inputs = Input(shape=(None,tag_size))
    
    glove_embedding = Embedding(vocab_size, glove_embedding_dim, #input_length=max_seq_len,
                                  weights=[glove_embedding_matrix],
                                  mask_zero=True,trainable=False)(inputs)
    
    word_embedding = Embedding(vocab_size, embedding_dim1, #input_length=max_seq_len, 
                               mask_zero=True, embeddings_regularizer=regularizers.l2(regularization))(inputs)
    
    #tag_embedding = Embedding(tag_size, tag_size, embeddings_regularizer=regularizers.l2(regularization))(tag_inputs)
    
    concat_embeds = Concatenate(axis=-1)([glove_embedding, word_embedding, tag_inputs])
    
    final_embed = Dense(units=embedding_dim2, activation='tanh',
                        kernel_regularizer=regularizers.l2(regularization))(concat_embeds)
    
    lstm1 = LSTM(hidden_dim1, activation='tanh', 
                   kernel_regularizer=regularizers.l2(regularization), 
                   recurrent_regularizer=regularizers.l2(regularization), #unroll=True, 
                   return_sequences = True, dropout=dropout_factor, recurrent_dropout=dropout_factor)(final_embed)
    
    lstm2 = LSTM(hidden_dim2, activation='tanh', 
                   kernel_regularizer=regularizers.l2(regularization), 
                   recurrent_regularizer=regularizers.l2(regularization), #unroll=True, 
                   return_sequences = True, dropout=dropout_factor, recurrent_dropout=dropout_factor)(lstm1)
    
    timedist_dropout = TimeDistributed(Dropout(dropout_factor))(lstm2)
    
    deep_dense = Dense(units=deeper_dim, activation='tanh', 
                       kernel_regularizer=regularizers.l2(regularization))(timedist_dropout)
    
    dropout_layer1 = Dropout(dropout_factor)(deep_dense)
    
    outputs = Dense(units=vocab_size, activation='softmax', 
                    kernel_regularizer=regularizers.l2(regularization))(dropout_layer1)
    
    model = Model(inputs=[inputs,tag_inputs], outputs=outputs)
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=learning_rate),
                  metrics=[sparse_categorical_crossentropy, sparse_categorical_accuracy], sample_weight_mode='temporal')
    return model

In [1]:
K.clear_session()
sess = tf.Session()
K.set_session(sess)

model = StackedLSTM(vocab_size=VOCAB_SIZE, glove_embedding_dim=GLOVE_EMBEDDING_DIM,
                    glove_embedding_matrix=glove_embedding_matrix, 
                    embedding_dim1=EMBEDDING_DIM1, embedding_dim2=EMBEDDING_DIM2,
                    hidden_dim1=HIDDEN_DIM1, hidden_dim2=HIDDEN_DIM2,
                    deeper_dim=DEEPER_DIM, max_seq_len=MAX_SEQ_LEN, dropout_factor=DROPOUT_FACTOR, 
                    regularization=REGULARIZATION, learning_rate=LEARNING_RATE, tag_size=TAG_SIZE)
print(model.summary())

NameError: name 'K' is not defined

In [24]:
class TB(TensorBoard):
    def __init__(self, log_every=1, **kwargs):
        super().__init__(**kwargs)
        self.log_every = log_every
        self.counter = 0
    
    def on_batch_end(self, batch, logs=None):
        self.counter+=1
        if self.counter%self.log_every==0:
            for name, value in logs.items():
                if name in ['batch', 'size']:
                    continue
                summary = tf.Summary()
                summary_value = summary.value.add()
                summary_value.simple_value = value.item()
                summary_value.tag = name
                self.writer.add_summary(summary, self.counter)
            self.writer.flush()
        
        super().on_batch_end(batch, logs)

In [ ]:
start_time = time()
tensorboard = TB(log_dir="./logs/" + MODEL_PREFIX + "/{}".format(time()), 
                          histogram_freq=0, write_graph=True, write_images=False, log_every=10)

callbacks=[tensorboard, 
           EarlyStopping(patience=5, monitor='val_loss'),
           ModelCheckpoint(filepath=MODELS_PATH + 'checkpoints/'+ MODEL_PREFIX + '_gen' + str(RUN_INDEX) + '.{epoch:02d}-{val_loss:.2f}.hdf5', 
                           monitor='val_loss', verbose=1, mode='auto', period=1), 
           ModelCheckpoint(filepath=MODELS_PATH + MODEL_PREFIX + '_gen'+str(RUN_INDEX)+'.hdf5', 
                           monitor='val_loss', verbose=1, mode='auto', period=1, save_best_only=True)]

model.fit([X_data, tag_data], y_data, epochs=10, batch_size=1024, shuffle=True, verbose=1, validation_split=0.2, callbacks=callbacks)

print("Total elapsed time: ", time()-start_time)

Train on 1333009 samples, validate on 333253 samples
Epoch 1/10
1333009/1333009 [==============================] - 968s 726us/step - loss: 5.4862 - sparse_categorical_crossentropy: 5.3589 - sparse_categorical_accuracy: 0.1469 - val_loss: 4.5909 - val_sparse_categorical_crossentropy: 4.3533 - val_sparse_categorical_accuracy: 0.2517

Epoch 00001: saving model to ./models/checkpoints/jokes_controlled_stacked_lstm_glove_gen1.01-4.59.hdf5

Epoch 00001: val_loss improved from inf to 4.59085, saving model to ./models/jokes_controlled_stacked_lstm_glove_gen1.hdf5
Epoch 2/10
1333009/1333009 [==============================] - 969s 727us/step - loss: 4.6670 - sparse_categorical_crossentropy: 4.3963 - sparse_categorical_accuracy: 0.2463 - val_loss: 4.3883 - val_sparse_categorical_crossentropy: 4.0964 - val_sparse_categorical_accuracy: 0.2820

Epoch 00002: saving model to ./models/checkpoints/jokes_controlled_stacked_lstm_glove_gen1.02-4.39.hdf5

Epoch 00002: val_loss improved from 4.59085 to 4.388

In [6]:
# generate a sequence from a language model
def generate_categorical(model, tokenizer, seed_text, maxlen, probabilistic=False, exploration_factor=1.0, tag=0):
    
    reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
    seq = tokenizer.texts_to_sequences([seed_text])[0]
    print(seq)
    
    while True:
        encoded_seq = seq
        if len(seq) > MAX_SEQ_LEN:
            encoded_seq = encoded_seq[-1*MAX_SEQ_LEN:]
            
        #padded_seq = pad_sequences([encoded_seq], maxlen=MAX_SEQ_LEN, padding='pre')
        padded_seq = np.array([seq])
        tags = to_categorical(np.full((1, padded_seq[0].shape[0]), tag), TAG_SIZE)
        y_prob = model.predict([padded_seq,tags])[0][-1].reshape(1,-1)#[3:].reshape(-1,1)
        
        if random.random() <= exploration_factor:
            probabilistic = True
        else:
            probabilistic = False
            
        if probabilistic:
            y_class = np.argmax(np.random.multinomial(1,y_prob[0]/(np.sum(y_prob[0])+1e-5),1))
        else:
            y_class = y_prob.argmax(axis=-1)[0]
        
        if y_class == 0:
            break
        out_word = reverse_word_map[y_class]
        seq.append(y_class)
        if out_word == 'eos' or len(seq) > maxlen or out_word == 'sos':
            break
    
    words = [reverse_word_map[idx] for idx in seq]
    
    return ' '.join(words)

In [8]:
K.clear_session()
sess = tf.Session()
K.set_session(sess)
model = load_model('models/jokes_controlled_stacked_lstm_glove_gen1.hdf5')
with open('models/jokes_controlled_stacked_lstm_glove_tokenizer_1.pickle', 'rb') as pickleFile:
    tokenizer = pickle.load(pickleFile)

In [10]:
print(sentences[:10])

['sos if god made everything threedots is god chinese ? eos', 'sos before you refer to someone as your ex , make sure they know you dated . eos', 'sos if you were arrested for masturbating on a plane threedots threedots they would have to charge you with hi - jacking eos', 'sos q : what did the blonde say when her doctor told her that she was pregnant ? a : " is it mine ? " eos', 'sos the past present and future threedots the past present and future walk into a bar . it was tense . eos', "sos knock knock who's there ? pill . pill who ? pill cosby . eos", "sos i went on one of those once in a lifetime holidays last week threedots i won't be doing that again . joke by tim vine eos", 'sos when your boss asks you " do i look stupid to you ? " it\'s a rhetorical question i know this now eos', "sos i ate an optimist once threedots but i couldn't keep him down . eos", 'sos reddit accounts should be treated like underwear . if you can keep them clean you could keep the same one for a while , b

In [19]:
MAX_SEQ_LEN = 13
TAG_SIZE = 2
joke = generate_categorical(model, tokenizer, "sos my boss likes", maxlen=40, tag=0, exploration_factor=0.1)
print(joke)

[1, 13, 500, 720]
sos my boss likes in the bathroom and i don't know what that working they are . eos


In [12]:
def bigrams_list(sentence):
    words = sentence.split(' ')
    bigrams = []
    for i in range(0, len(words)-1):
        bigrams.append(words[i]+' '+words[i+1])
    return bigrams

print(bigrams_list("sos hello , i'm a dinosaur . eos"))

['sos hello', 'hello ,', ", i'm", "i'm a", 'a dinosaur', 'dinosaur .', '. eos']


In [17]:
sentence_bigrams = [bigrams_list(s) for s in sentences]
print(sentence_bigrams[:2])

[["sos what's", "what's the", 'the difference', 'difference between', 'between a', 'a strawberry', 'strawberry and', 'and a', 'a pencil', 'pencil ?', "? one's", "one's a", 'a fruit', 'fruit ,', ', you', 'you idiot', 'idiot .', '. eos'], ['sos my', 'my bank', 'bank says', 'says my', 'my password', "password isn't", "isn't strong", 'strong enough', 'enough .', '. did', 'did it', 'it ever', 'ever stop', 'stop and', 'and think', 'think that', 'that my', 'my password', 'password has', 'has a', 'a lot', 'lot going', 'going on', 'on right', 'right now', 'now ?', '? eos']]


In [18]:
def intersection(lst1, lst2):
    temp = set(lst2)
    lst3 = [value for value in lst1 if value in temp]
    return lst3

def similarity_score(lst1, lst2):
    intersection_len = len(intersection(lst1, lst2))
    return (1.0*intersection_len)/len(lst1)#+len(lst2)-intersection_len)
 
def print_closest_sentences(sentence, sentence_bigrams, top_k=3):
    bigrams = bigrams_list(sentence)
    scores = np.array([similarity_score(bigrams, sbigrams)
                       for sbigrams in sentence_bigrams])
    top_k_indices = scores.argsort()[-1*top_k:][::-1]
    top_k_scores = scores[top_k_indices]
    for k in range(top_k):
        print(top_k_scores[k], " -> ", sentences[top_k_indices[k]])

In [19]:
print_closest_sentences(joke, sentence_bigrams, 10)

0.16666666666666666  ->  sos i really want to make a period joke . but it's just not that time of the month for me eos
0.1111111111111111  ->  sos " was i adopted ? " " yes . but they brought you back . " eos
0.1111111111111111  ->  sos why did the blonde freeze a pot of boiling water ? because you never know when you're going to need some boiling water . eos
0.1111111111111111  ->  sos a good time to keep your mouth shut is when you're in deep water . eos
0.1111111111111111  ->  sos i better have a baby soon before my mom gets too old to raise it for me eos
0.1111111111111111  ->  sos when you're sad , hug a kid . but make sure it's yours cuz that shit would be weird . eos
0.1111111111111111  ->  sos if i'm ever being chased by a giraffe i'm gonna run into a place with low ceiling fans sorry giraffe but i gotta do what's best for me eos
0.1111111111111111  ->  sos he was a real gentlemen and always opened the fridge door for me eos
0.1111111111111111  ->  sos i wanted to see lot of an

In [25]:
joke = generate_categorical(model, tokenizer, "eos baby a off", maxlen=40, tag=1)
joke = ' '.join(joke.split(' ')[::-1])
print(joke)
print_closest_sentences(joke, sentence_bigrams, 10)

[2, 195, 4, 110]
sos i wanted a coming elton so hard to neighbors , " i'm not having god " so he engineers off a baby eos
0.17391304347826086  ->  sos i crashed into a dwarf at some traffic lights he got out of his car and said " i'm not happy . " i replied , " well , which one are you then ? " eos
0.17391304347826086  ->  sos i had a parrot that talked . . . but it never said , " i'm hungry . " so it died . eos
0.17391304347826086  ->  sos i texted my boss , " what's the difference between this morning and your daughter ? " he answered , " i don't know . " i replied , " i'm not coming in this morning . " eos
0.17391304347826086  ->  sos i had a parrot . the parrot talked , but it did not say " i'm hungry , " so it died . eos
0.17391304347826086  ->  sos i told the barber , " a little off the top . " so he gave me a circumcision . eos
0.17391304347826086  ->  sos i rang up a local builder and said , " i want a skip outside my house . " he said , " i'm not stopping you . " * tim vine * 

In [22]:
joke = generate_categorical(model, tokenizer, "eos . sport contact", maxlen=40, tag=1)
joke = ' '.join(joke.split(' ')[::-1])
print(joke)
print_closest_sentences(joke, sentence_bigrams, 10)

[2, 3, 2082, 1698]
sos what do you call a bunch of sauce on a plane ? the whiskey contact sport . eos
0.5555555555555556  ->  sos what do you call a bunch of male pornstars on a flight together ? snakes on a plane . eos
0.5555555555555556  ->  sos what do you call a bunch of white guys sitting on a bench ? the nba . eos
0.5  ->  sos what do you call a black man flying a plane ? the pilot you racist fuck . eos
0.5  ->  sos what do you call a middle eastern man flying a plane ? the pilot , you racist . eos
0.5  ->  sos what do you call a bunch of white guys on a bench ? the nba eos
0.5  ->  sos what do you call a bunch of white guys sitting on a bench ? the nba eos
0.5  ->  sos what do you call a gay guy flying a plane ? the pilot . eos
0.5  ->  sos what do you call a muslim on a plane ? a passenger threedots you racist bastard . eos
0.5  ->  sos what do you call a black man flying a plane ? the pilot . eos
0.5  ->  sos what do you call a bunch of white dudes sitting on a bench ? the nba

In [43]:
joke = generate_categorical(model, tokenizer, "sos i am not going", maxlen=40, tag=0, exploration_factor=0.3)
print(joke)
print_closest_sentences(joke, sentence_bigrams, 10)

[1, 7, 237, 58, 121]
sos i am not going to be a comedian . i just say " i don't know what i know , but i don't know what , but i don't know what i think about . " eos
0.40540540540540543  ->  sos i bought some shoes from my pill dealer on friday . i don't know what he laced them with , but i was tripping all weekend . eos
0.40540540540540543  ->  sos i bought shoes from a drug dealer once . i don't know what he laced them with , but i was tripping all day . eos
0.40540540540540543  ->  sos i don't know what you do when you come across a bear , but i just wipe it off and apologize . eos
0.3783783783783784  ->  sos i bought shoes from a drug dealer i don't know what he laced them with , but i was tripping all day eos
0.3783783783783784  ->  sos this is the last pair of shoes i buy from a drug dealer threedots i don't know what he laced them with , but i just keep tripping . eos
0.3783783783783784  ->  sos i bought shoes off a drug dealer i don't know what he laced them with , but i was t